In [2]:
pip install tensorflow

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl (390.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\TAZ78\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\TAZ78\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from random import randint
import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model



In [ ]:
#Load the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,
224, 3))


In [ ]:
#Freeze the Layers
for layer in base_model.layers:
 layer.trainable = False


In [ ]:
#Add Custom Layers
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x) # For binary classification
model = Model(inputs=base_model.input, outputs=x)

#Ensure input matches Data Generators output
input_shape = (224,224,3) #Assuming you use RGB Images
model.build(input_shape=(None, *input_shape)) #Build model with explicit input shape


In [ ]:
#Choose a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
 loss='binary_crossentropy',
 metrics=['accuracy'])


In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

In [ ]:
#Load Training and Validation Data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='binary'
)
val_data = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
#Train the Model
history = model.fit(train_data, epochs=5, validation_data=val_data)


In [ ]:
#Fine-Tune it
for layer in model.layers[-4:]: # Unfreeze last 4 layers
 layer.trainable = True
# Re-compile and train
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
 loss='binary_crossentropy',
 metrics=['accuracy'])
history = model.fit(train_data, epochs=5, validation_data=val_data)
